In [102]:
import urllib.request
from bs4 import BeautifulSoup
import string
import pandas as pd

In [4]:
url="https://www.basketball-reference.com/players/"

In [184]:
alphabet=string.ascii_lowercase
playerURL=[]
playerID=[]
playerName=[]
yearStart=[]
yearEnd=[]
position=[]
height=[]
weight=[]
birthday=[]
college=[]

In [185]:
for i in range(0,26):
    playerURL.append(url+alphabet[i])
del playerURL[23]

In [186]:
for i in range(0,25):
    x = urllib.request.urlopen(playerURL[i]).read()
    soup = BeautifulSoup(x, "lxml")
    playerTag=soup.findAll(attrs={"data-stat" : "player"})
    YearMinTag=soup.findAll(attrs={"data-stat" : "year_min"})
    YearMaxTag=soup.findAll(attrs={"data-stat" : "year_max"})
    positionTag=soup.findAll(attrs={"data-stat" : "pos"})
    heightTag=soup.findAll(attrs={"data-stat" : "height"})
    weightTag=soup.findAll(attrs={"data-stat" : "weight"})
    bdayTag=soup.findAll(attrs={"data-stat" : "birth_date"})
    collegeTag=soup.findAll(attrs={"data-stat" : "college_name"})

    for i in range(1, len(playerTag)):
        playerID.append(playerTag[i].attrs["data-append-csv"])
        playerName.append(playerTag[i].a.contents[0])
        yearStart.append(YearMinTag[i].contents[0])
        yearEnd.append(YearMaxTag[i].contents[0])
        if(not positionTag[i].contents):
            position.append("")
        else:
            position.append(positionTag[i].contents[0])
        if(not heightTag[i].contents):
            height.append("")
        else:
            height.append(heightTag[i].contents[0])
        if(not weightTag[i].contents):
            weight.append("")
        else:
            weight.append(weightTag[i].contents[0])
        if(not bdayTag[i].contents):
            birthday.append("")
        else:
            birthday.append(bdayTag[i].a.contents[0])
        if(not collegeTag[i].contents):
            college.append("")
        else:
            college.append(collegeTag[i].a.contents[0])


In [187]:
labels = ['Player_ID', 'Name', 'YrStart', 'YrEnd', 'Pos', 'Height', 'Weight', 'Birth_date', 'College']
player_df = pd.DataFrame({"Player_ID": playerID,
                         "Name": playerName,
                         "YrStart": yearStart,
                         "YrEnd": yearEnd,
                         "Pos": position,
                         "Height": height,
                         "Weight": weight,
                         "Birth_date" : birthday,
                         "College" : college})
player_df=player_df[labels]


In [189]:
player_df.to_csv("NBA_Players.csv")